In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras

import cv2
import numpy as np

In [2]:
video_ids = [2] # range(1, 6)
frame_skip = 8

In [3]:
labels = []
for id in video_ids:
    with open(f'data/labels/video{id}.txt') as f:
        lines = f.readlines()
        lines = [line.split() for line in lines]
        lines = np.array(lines)

        # Remove OTH label
        lines = lines[lines[:, 1] != 'OTH']
        
        # Prune frames
        lines = lines[::frame_skip]

        labels.append(lines)

labels = np.array(labels)

# Show frequency of labels
# unique, counts = np.unique(labels[:, 1], return_counts=True)
# import matplotlib.pyplot as plt
# plt.bar(unique, counts)
# plt.show()

In [4]:
# Load model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

In [5]:
data = []
for id, labels in zip(video_ids, labels):
    video = cv2.VideoCapture(f"data/videos/video{id}.mp4")
    for i, frame_num in enumerate(labels[:, 0].astype(int)):
        print(f'Loading video {id}... ({i}/{labels.shape[0]})', end='\r')
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = video.read()

        input_img = tf.expand_dims(frame, axis=0)
        input_img = tf.image.resize_with_pad(input_img, 192, 192)
        input_img = tf.cast(input_img, dtype=tf.int32)

        # Detection section
        keypoints_with_scores = movenet(input_img)['output_0'].numpy().flatten()
    
        data.append(keypoints_with_scores)

    video.release()

data = np.array(data)

In [6]:
# One-hot encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(labels[:, 1])
labels = label_encoder.transform(labels[:, 1])

labels = keras.utils.to_categorical(labels)

In [12]:
from keras.layers import LSTM, Dense, LayerNormalization

# model = tf.keras.models.Sequential([
#     LSTM(128, input_shape=(17*3, 1)),
#     LayerNormalization(),
#     Dense(10, activation='relu'),
# ])

model = tf.keras.models.Sequential([
    LSTM(64, return_sequences=True, activation="relu",input_shape=(17*3, 1)),
    LayerNormalization(axis=1),
	LSTM(32, return_sequences=False, activation="relu",input_shape=(17*3, 1)),
	LayerNormalization(axis=1),
    Dense(32, activation='relu'),
	Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 51, 64)            16896     
                                                                 
 layer_normalization_7 (Laye  (None, 51, 64)           102       
 rNormalization)                                                 
                                                                 
 lstm_8 (LSTM)               (None, 32)                12416     
                                                                 
 layer_normalization_8 (Laye  (None, 32)               64        
 rNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 10)               

In [13]:
import datetime
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
# Add Early Stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.fit(X_train, y_train, epochs=250, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])


!tensorboard --logdir logs

Epoch 1/250
44/44 [==============================] - 4s 49ms/step - loss: 2.0964 - accuracy: 0.2030 - val_loss: 2.0075 - val_accuracy: 0.2407
Epoch 2/250
44/44 [==============================] - 2s 42ms/step - loss: 2.0185 - accuracy: 0.2310 - val_loss: 1.9825 - val_accuracy: 0.2722
Epoch 3/250
44/44 [==============================] - 2s 43ms/step - loss: 1.9894 - accuracy: 0.2296 - val_loss: 1.9463 - val_accuracy: 0.2751
Epoch 4/250
44/44 [==============================] - 2s 41ms/step - loss: 1.9514 - accuracy: 0.2597 - val_loss: 1.9074 - val_accuracy: 0.2722
Epoch 5/250
44/44 [==============================] - 2s 44ms/step - loss: 1.8937 - accuracy: 0.2841 - val_loss: 1.9472 - val_accuracy: 0.2693
Epoch 6/250
44/44 [==============================] - 2s 41ms/step - loss: 1.8383 - accuracy: 0.2991 - val_loss: 1.8307 - val_accuracy: 0.3295
Epoch 7/250
44/44 [==============================] - 2s 44ms/step - loss: 1.8094 - accuracy: 0.3135 - val_loss: 1.8045 - val_accuracy: 0.3496
Epoch 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

model.save("your_model.h5")